In [1]:
!pip install -q -U protobuf qdrant-client sentence-transformers openai anthropic langchain langchain-community

In [26]:
import json
from typing import List, Dict
import qdrant_client
from qdrant_client.models import VectorParams, Distance
from sentence_transformers import SentenceTransformer
import openai
import anthropic
from qdrant_client import QdrantClient
from qdrant_client.http import models
from openai import OpenAI
import os

## Scenario Overview

Imagine an e-commerce website that has an extensive FAQ page. This FAQ page is designed to help users by answering common questions about products, shipping, returns, payment methods, and more. However, users often prefer to get their answers directly through an interactive chat system rather than manually searching through the FAQ page.

## Objective
The goal is to build a Retrieval-Augmented Generation (RAG) system that can directly interact with users and provide them with accurate and relevant answers by leveraging the existing FAQ data. This means that instead of having the user navigate through the FAQ page, the system will automatically retrieve the most relevant information from the FAQ dataset and present it in a conversational format.

## How It Works

- **FAQ Dataset**: We have a dataset consisting of JSON objects, each containing a frequently asked question (FAQ) and its corresponding answer. This dataset is sourced from Kaggle and is specifically designed for e-commerce-related queries: [Ecommerce-FAQ-Chatbot-Dataset](https://www.kaggle.com/datasets/saadmakhdoom/ecommerce-faq-chatbot-dataset)

- **User Interaction**: A user asks a question through the chat interface. Instead of redirecting the user to the FAQ page, the system will process the question in real-time.

- **Retrieval Mechanism**: The system will search through the FAQ dataset to find the most relevant question-answer pair(s) that match the user's query.

- **Answer Generation**: Once the relevant information is retrieved, the system will generate a response that is directly presented to the user in a conversational manner. The response may be the exact answer from the dataset or a synthesized answer that is tailored to the user's query.

## Define the classes

In [48]:
# Load the dataset (JSON file)
def load_faq_data(file_path: str) -> Dict[str, str]:
    with open(file_path, 'r') as file:
        return json.load(file)['questions']

# Embedding model (sentence transformer)
class EmbeddingModel:
    def __init__(self, model_name: str = 'all-MiniLM-L6-v2'):
        self.model = SentenceTransformer(model_name)

    def get_embeddings(self, sentences: List[str]) -> List[List[float]]:
        return self.model.encode(sentences).tolist()

# Qdrant vector store client
class QdrantVectorStore:
    def __init__(self, collection_name: str):
        self.client = QdrantClient(":memory:")
        self.collection_name = collection_name

    def create_collection(self, vector_size: int):
        if self.collection_name in self.client.get_collections().collections:
            print(f"Collection '{self.collection_name}' already exists. Skipping creation.")
        else:
            self.client.create_collection(
                collection_name=self.collection_name,
                vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE)
            )

    def add_documents(self, documents: List[Dict[str, str]], embeddings: List[List[float]]):
        self.client.upsert(
            collection_name=self.collection_name,
            points=qdrant_client.models.Batch(
                ids=[i for i in range(len(documents))],
                vectors=embeddings,
                payloads=documents
            )
        )

    def search(self, query_vector: List[float], limit: int = 3) -> List[Dict]:
        results = self.client.search(
            collection_name=self.collection_name,
            query_vector=query_vector,
            limit=limit
        )
        return [hit.payload for hit in results]

# Retriever
class Retriever:
    def __init__(self, embedding_model: EmbeddingModel, vector_store: QdrantVectorStore):
        self.embedding_model = embedding_model
        self.vector_store = vector_store

    def retrieve(self, query: str, limit: int = 3) -> List[Dict]:
        query_embedding = self.embedding_model.get_embeddings([query])[0]
        return self.vector_store.search(query_embedding, limit)

# LLM (GPT 3.5 turbo)
class LLM:
    def __init__(self, api_key: str):
        openai.api_key = api_key

    def generate_response(self, query: str, context: List[Dict]) -> str:
        context_str = "\n".join([f"Q: {item['question']}\nA: {item['answer']}" for item in context])
        prompt = f"Given the following FAQ context:\n\n{context_str}\n\nAnswer the following question: {query}\n\nAnswer:"

        response = openai.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that answers questions based on the given FAQ context."},
                {"role": "user", "content": prompt}
            ]
        )

        return response.choices[0].message.content.strip()

class RAGSystem:
    def __init__(self, retriever: Retriever, llm: LLM):
        self.retriever = retriever
        self.llm = llm

    def answer_question(self, query: str) -> str:
        relevant_docs = self.retriever.retrieve(query)
        return self.llm.generate_response(query, relevant_docs)

## Load the dataset (JSON file)

In [30]:
faq_data = load_faq_data("Ecommerce_FAQ_Chatbot_dataset.json")
embedding_model = EmbeddingModel()

In [5]:
print(faq_data)

[{'question': 'How can I create an account?', 'answer': "To create an account, click on the 'Sign Up' button on the top right corner of our website and follow the instructions to complete the registration process."}, {'question': 'What payment methods do you accept?', 'answer': 'We accept major credit cards, debit cards, and PayPal as payment methods for online orders.'}, {'question': 'How can I track my order?', 'answer': "You can track your order by logging into your account and navigating to the 'Order History' section. There, you will find the tracking information for your shipment."}, {'question': 'What is your return policy?', 'answer': 'Our return policy allows you to return products within 30 days of purchase for a full refund, provided they are in their original condition and packaging. Please refer to our Returns page for detailed instructions.'}, {'question': 'Can I cancel my order?', 'answer': 'You can cancel your order if it has not been shipped yet. Please contact our cus

## Create a collection in the vector store

In [31]:
collection_name = "extractive-question-answering"

vector_store = QdrantVectorStore(collection_name)

client = vector_store.client

collections = client.get_collections()
print(collections)

# only create collection if it doesn't exist
if collection_name not in [c.name for c in collections.collections]:
    client.recreate_collection(
        collection_name=collection_name,
        vectors_config=models.VectorParams(
            size=384,
            distance=models.Distance.COSINE,
        ),
    )
collections = client.get_collections()
print(collections)

collections=[]
collections=[CollectionDescription(name='extractive-question-answering')]


<ipython-input-31-8d4a74110ac6>:12: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


## Add the embeddings corresponding to the answers of the FAQ dataset

The system primarily focuses on embedding the answers from the FAQ dataset, as they contain the most critical information. However, concatenating the questions and answers and then embedding them can be an alternative approach if the questions provide additional context. If there is no existing FAQ dataset, web scraping is employed to collect the necessary data from the website, after which the same embedding process is applied.

In [32]:
embeddings = embedding_model.get_embeddings([item['answer'] for item in faq_data])

vector_store.add_documents(faq_data, embeddings)

retriever = Retriever(embedding_model, vector_store)

## Define the LLM and the RAG system

In [49]:
llm = LLM(api_key='OPENAI_API_KEY')
rag_system = RAGSystem(retriever, llm)

### Test

In [51]:
example_queries = [
    "What is your return policy?",
    "How long does shipping usually take?",
    "Do you offer international shipping?",
    "Can I change or cancel my order after it's been placed?",
    "What payment methods do you accept?"
]

for query in example_queries:
    answer = rag_system.answer_question(query)
    print(f"\nQuestion: {query}")
    print(f"Answer: {answer}")
    print("-" * 50)


Question: What is your return policy?
Answer: Our return policy allows you to return products within 30 days of purchase for a full refund, provided they are in their original condition and packaging. Please refer to our Returns page for detailed instructions.
--------------------------------------------------

Question: How long does shipping usually take?
Answer: Shipping usually takes 3-5 business days for standard shipping and 1-2 business days for express shipping.
--------------------------------------------------

Question: Do you offer international shipping?
Answer: Yes, we offer international shipping to select countries.
--------------------------------------------------

Question: Can I change or cancel my order after it's been placed?
Answer: You can only change or cancel your order after it's been placed if it has not been shipped yet. Please contact our customer support team with your order details, and we will assist you with the necessary steps.
----------------------